In [5]:
import pandas as pd
import time
import nltk
from nltk.tokenize import word_tokenize
import torch
import numpy as np
import transformers
stopwords = nltk.corpus.stopwords.words('english')

In [11]:
df = pd.read_csv('data/dataset/can_statements.csv', index_col=0)
df.dropna(inplace=True)

In [26]:
filtered_tokens = [t for t in word_tokenize(df.text[0]) if t not in stopwords] 
def tokenize(text):
    tok = [t for t in word_tokenize(text) if t not in stopwords and not t.isnumeric()]
    sentence = ''
    for word in tok:
        word = word if len(word) > 1 else ''
        word = '' if word.isnumeric() else word
        word = word.lower()
        
        if word != '':
            sentence += word + ' '
    return sentence[:500]

df['text'] = df['text'].apply(tokenize)
df.iloc[1]['text']

'bank canada today increased target overnight rate bank rate deposit rate bank also continuing policy quantitative tightening global canadian economies evolving broadly line bank july projection effects covid-19 outbreaks ongoing supply disruptions war ukraine continue dampen growth boost prices global inflation remains high measures core inflation moving countries response central banks around world continue tighten monetary policy economic activity united states moderated although us labour mar'

In [24]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import scipy.special

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def sentiment(input):
    inputs = tokenizer(input, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    scores = {k: v for k, v in zip(model.config.id2label.values(), scipy.special.softmax(logits.numpy().squeeze()))}
    return scores['positive'] - scores['negative']
# {'negative': 0.034473564, 'neutral': 0.067165166, 'positive': 0.8983614}

In [27]:
df['sentiment'] = df['text'].apply(sentiment)

In [29]:
df.to_csv('data/dataset/can_statements_sentiment.csv')

In [30]:
df.describe()

,sentiment
count,359.000000
mean,0.092986
std,0.446087
min,-0.947138
25%,-0.023326
50%,0.068691
75%,0.324349
max,0.925003
